#### Creación de Tablas

In [1]:
import Tablas.Campus as Campus, Tablas.Carreras as Carreras, Tablas.Ciudades as Ciudades, Tablas.Municipios as Municipios, Tablas.Facultades as Facultades, Tablas.Paises as Paises, Tablas.Personas_carreras as Personas_carreras, Tablas.Personas as Personas, Tablas.Programas as Programas, Tablas.Provincias as Provincias, Tablas.Ramas as Ramas, Tablas.Tipo_persona as Tipo_persona, Tablas.Ubicaciones as Ubicaciones, Tablas.Genero as Genero
from sqlalchemy import MetaData
from Configuracion_BD import crear_conexion, obtener_sesion, Base
import pandas as pd

meta = MetaData()	

base_general = "pil_trabajo_practico"

engine_mysql = crear_conexion("mysql","root","Contadores2","127.0.0.1:4000",base_general)

session_mysql = obtener_sesion(engine_mysql)

Base.metadata.create_all(engine_mysql) # creación de las tablas

#### Lectura de los datos contenidos en los archivos csv y generación de los dataframe

In [2]:
profesoresDF = pd.read_csv("Bases de datos/Profesores.csv")
profesoresDF["tipopersona"]="profesor" # Agregamos esta nueva columna con la constante profesor en el tipo de persona para este DF

alumnosDF = pd.read_csv("Bases de datos/Alumnos.csv")
alumnosDF["tipopersona"]="alumno" # Agregamos esta nueva columna con la constante alumno en el tipo de persona para este DF

cursos_profesoresDF = pd.read_csv("Bases de datos/cursos_profesores.csv")

personasDF = pd.concat([profesoresDF, alumnosDF]) # Fusionamos los dataframe de alumnos y profesores

#### Generación de los dataset 

In [3]:
def generar_dataset(DF):
	dataset=[]
	for index, fila in DF.iterrows():
		dataset.append({**fila})
	return dataset
	
dataset_personas= generar_dataset(personasDF)
dataset_alumnos= generar_dataset(alumnosDF)
dataset_cursos_profesores= generar_dataset(cursos_profesoresDF)

#### Extraer valores únicos del campo "gender" del dataset Personas y traducirlos al español

In [4]:
valores_unicos=[]
for x in dataset_personas: # recorremos el dataset
    if x["gender"] not in valores_unicos: # si el el valor de gender no está en la lista, lo agregamos a dicha lista 
            valores_unicos.append(x["gender"])    # de esta forma se obtienen los valores únicos que se utilizan a continuación

for x in dataset_personas:
    if x["gender"]=="Female":
        x["gender"]="Femenino"
    elif x["gender"]=="Male":
        x["gender"]="Masculino"
    elif x["gender"]=="Non-binary":
        x["gender"]="No-binario"
    elif x["gender"]=="Polygender":
        x["gender"]="Poligénero"
    elif x["gender"]=="Genderqueer":
        x["gender"]="Género queer"
    elif x["gender"]=="Agender":
        x["gender"]="Agénero"
    elif x["gender"]=="Bigender":
        x["gender"]="Bigénero"
    elif x["gender"]=="Genderfluid":
        x["gender"]="Género fluido" 

#### Convertir los campos "personal_id" e "instructor" de str a número

In [5]:
def conversion_dni_str_numero(dataset): # función para convertir el formato de los campos personal_id e instructor, que serían el dni, de str a número
    for x in dataset: # recorremos el dataset
        if "personal_id" in x: # seleccionamos la clave personal_id                  
            j=x["personal_id"].replace('.','') # eliminamos los puntos mediante el método replace
            x["personal_id"]=j # asignamos un nuevo valor a la clave personal_id
        if "instructor" in x: # lo mismo pero con el campo instructor
            j=x["instructor"].replace('.','')
            x["instructor"]=j    
    
conversion_dni_str_numero(dataset_cursos_profesores)
conversion_dni_str_numero(dataset_alumnos)
conversion_dni_str_numero(dataset_personas)

#### Insertar datos en tablas a partir del dataset Personas

In [6]:
from sqlalchemy import and_

lista_errores_personas= []

for fila in dataset_personas:
	session_mysql.begin() # Iniciamos la sesión
	try:
		
		# --------------- Tabla Paises ------------------------------------

		pais = session_mysql.query(Paises.Paises).filter(Paises.Paises.nombre == fila['country']).first()
		if pais == None : # Con esto sólo creamos un registro si no existe previamente, con lo cual evitamos la repetición de los datos
			pais = Paises.Paises(nombre=fila['country']) # se crea el objeto
			session_mysql.add(pais) # se lo añade, quedándo en la memoria del orm

		# --------------- Tabla Ciudades ------------------------------------

		ciudad = session_mysql.query(Ciudades.Ciudades).filter(Ciudades.Ciudades.nombre == fila['city']).first()
		if ciudad == None:
			ciudad = Ciudades.Ciudades(nombre=fila['city'])
			session_mysql.add(ciudad)

		# --------------- Tabla Municipios ------------------------------------

		municipio = session_mysql.query(Municipios.Municipios).filter(Municipios.Municipios.nombre == fila['town']).first()
		if municipio == None:
			municipio = Municipios.Municipios(nombre=fila['town'])
			session_mysql.add(municipio)

		# --------------- Tabla Provincias ------------------------------------

		provincia = session_mysql.query(Provincias.Provincias).filter(Provincias.Provincias.nombre == fila['state']).first()
		if provincia == None:
			provincia = Provincias.Provincias(nombre=fila['state'])
			session_mysql.add(provincia)

		# --------------- Tabla Ubicaciones ------------------------------------

		ubicacion = session_mysql.query(Ubicaciones.Ubicaciones).filter(and_ (Ubicaciones.Ubicaciones.pais_id == pais.id , Ubicaciones.Ubicaciones.municipio_id == municipio.id , Ubicaciones.Ubicaciones.provincia_id == provincia.id , Ubicaciones.Ubicaciones.ciudad_id == ciudad.id)).first()
		if ubicacion == None:
			ubicacion = Ubicaciones.Ubicaciones(pais=pais, municipio=municipio, provincia=provincia, ciudad=ciudad)
			session_mysql.add(ubicacion)	

		# --------------- Tabla Genero ------------------------------------

		genero = session_mysql.query(Genero.Genero).filter(Genero.Genero.nombre == fila['gender']).first()
		if genero == None :
			genero = Genero.Genero(nombre=fila['gender'])
			session_mysql.add(genero)

		# --------------- Tabla Tipo_persona ------------------------------------

		tipo_persona = session_mysql.query(Tipo_persona.Tipo_Persona).filter(Tipo_persona.Tipo_Persona.nombre == fila['tipopersona']).first()
		if tipo_persona == None :
			tipo_persona = Tipo_persona.Tipo_Persona(nombre=fila['tipopersona'])
			session_mysql.add(tipo_persona)

		# --------------- Tabla Personas ------------------------------------		
		
		# La persona se inserta al final porque se necesitan las entidades de género, tipo_persona y ubicación ya cargadas
		persona = session_mysql.query(Personas.Personas).filter(Personas.Personas.personal_id == fila['personal_id'], ).first()
		if persona == None :
			persona = Personas.Personas( nombre = fila['first_name'], apellido = fila['last_name'], email = fila['email'], birthdate = fila['birthdate'], personal_id = fila['personal_id'],
							tipo_persona = tipo_persona, ubicacion = ubicacion, genero = genero)
			session_mysql.add(persona)

		session_mysql.commit() # Finalmente hacemos el commit de todos los registros añadidos para insertarlos en la base
	
	except Exception as error:
		print(error) 
		session_mysql.rollback() # si ocurre algún error, hacemos un rollback revirtiendo todos los datos que hayan sido cargados de un registro en particular
		lista_errores_personas.append(fila) # agregamos el registro que produjo el error a una lista, para poder analizarlo después
	
	finally: 
		session_mysql.close()
		engine_mysql.dispose()

#### Insertar datos en tablas a partir del dataset cursos_profesores

In [7]:
lista_errores_cursos_profesores=[]

for fila in dataset_cursos_profesores:
	session_mysql.begin()
	try:
		
		# --------------- Tabla Programas ------------------------------------	
				
		programa = session_mysql.query(Programas.Programas).filter(Programas.Programas.nombre == fila['program']).first()
		if programa == None:
			programa = Programas.Programas(nombre=fila['program'])
			session_mysql.add(programa)

		# --------------- Tabla Facultades ------------------------------------	

		facultad = session_mysql.query(Facultades.Facultades).filter(Facultades.Facultades.nombre == fila['institute']).first()
		if facultad == None:
			facultad = Facultades.Facultades(nombre=fila['institute'])
			session_mysql.add(facultad)

		# --------------- Tabla Ramas ------------------------------------	

		rama = session_mysql.query(Ramas.Ramas).filter(Ramas.Ramas.nombre == fila['branch']).first()
		if rama == None:
			rama = Ramas.Ramas(nombre=fila['branch'])
			session_mysql.add(rama)

		# --------------- Tabla Campus ------------------------------------		

		campus = session_mysql.query(Campus.Campus).filter(Campus.Campus.nombre == fila['campus']).first()
		if campus == None:
			campus = Campus.Campus(nombre=fila['campus'])
			session_mysql.add(campus)

		# --------------- Tabla Carreras ------------------------------------		

		carrera = session_mysql.query(Carreras.Carreras).filter(and_(Carreras.Carreras.campus_id==campus.id,Carreras.Carreras.facultad_id==facultad.id,Carreras.Carreras.rama_id==rama.id,Carreras.Carreras.programa_id==programa.id)).first()
		if carrera == None:
			carrera = Carreras.Carreras(programa=programa,facultad=facultad,rama=rama,campus=campus) 
			session_mysql.add(carrera)

		# --------------- Tabla Personas_carreras (profesores) ------------------------------------			
		
		profesor= session_mysql.query(Personas.Personas).filter(Personas.Personas.personal_id==fila['instructor']).first()		
		profesores_carreras=session_mysql.query(Personas_carreras.Personas_Carreras).filter(and_(Personas_carreras.Personas_Carreras.persona_id==profesor.id,Personas_carreras.Personas_Carreras.carrera_id==carrera.id)).first()
		if profesores_carreras == None:
			profesor_carrera = Personas_carreras.Personas_Carreras(persona_id=profesor.id,carrera_id=carrera.id)			
			session_mysql.add(profesor_carrera)

		session_mysql.commit()

	except Exception as error:
		print(error)		
		session_mysql.rollback()
		lista_errores_cursos_profesores.append(fila)
		
	finally:
		session_mysql.close()
		engine_mysql.dispose()

In [8]:
lista_errores_alumnos=[]

for fila in dataset_alumnos:
	session_mysql.begin()
	try:

		# --------------- Tabla Personas_carreras (alumnos) ------------------------------------			

		alumno= session_mysql.query(Personas.Personas).filter(Personas.Personas.personal_id==fila['personal_id']).first()
		carrera= session_mysql.query(Carreras.Carreras).join(Programas.Programas).filter(Programas.Programas.nombre==fila['program']).first()
		alumnos_carreras=session_mysql.query(Personas_carreras.Personas_Carreras).filter(and_(Personas_carreras.Personas_Carreras.persona_id==alumno.id,
		Personas_carreras.Personas_Carreras.carrera_id==carrera.id)).first()
		if alumnos_carreras == None:
			alumno_carrera = Personas_carreras.Personas_Carreras(persona_id=alumno.id,carrera_id=carrera.id)
			session_mysql.add(alumno_carrera)
		session_mysql.commit()
	
	except Exception as error:
		print(error)		
		session_mysql.rollback()
		lista_errores_alumnos.append(fila)
	
	finally:
		session_mysql.close()
		engine_mysql.dispose()